In [1]:
import numpy as np
import matplotlib.pyplot as plt
import galpy as gp
from galpy.orbit import Orbit
from galpy.potential import *
from astropy import units as u
from copy import deepcopy
import wurlitzer
from galpy.util import bovy_conversion
import matplotlib as mpl
from scipy import stats
from matplotlib.colors import LinearSegmentedColormap
import pickle

In [2]:
%load_ext wurlitzer

In [3]:
%matplotlib nbagg

# Define all dwarf galaxies

In [4]:
ts = np.linspace(0., -12/bovy_conversion.time_in_Gyr(ro=8.,vo=220.), 1201)
dt = (ts[1]-ts[0])/250

# SAGITTARIUS
sgr_M = 1.4E10*u.Msun
sgr_a = 7*u.kpc # 6.5kpc from https://arxiv.org/pdf/1608.04743.pdf
sgr_pot = HernquistPotential(amp=sgr_M, a=sgr_a, ro=8., vo=220.)
sgr_orbit = Orbit(vxvv=[283.8313, -30.5453, 26.67, -2.692, -1.359, 140], radec=True, ro=8.,vo=220.) 
sgr_orbit.integrate(ts, MWPotential2014, dt=dt)
sgr_potential = MovingObjectPotential(sgr_orbit, pot=sgr_pot)

# LMC
lmc_M = 1E11*u.Msun # https://arxiv.org/pdf/1608.04743.pdf
lmc_a = 10.2*u.kpc  # https://iopscience.iop.org/article/10.1088/2041-8205/721/2/L97/pdf
lmc_pot = HernquistPotential(amp=lmc_M, a=lmc_a, ro=8., vo=220.)
lmc_orbit = Orbit(vxvv=[78.77, -69.01, 50.1, 1.850, 0.234, 262.2], radec=True, ro=8.,vo=220.) 
lmc_orbit.integrate(ts, MWPotential2014, dt=dt)
lmc_potential = MovingObjectPotential(lmc_orbit, pot=lmc_pot)

# SMC
smc_M = 2.6E10*u.Msun # https://iopscience.iop.org/article/10.1088/2041-8205/721/2/L97/pdf
smc_a = 3.6*u.kpc     # https://iopscience.iop.org/article/10.1088/2041-8205/721/2/L97/pdf
smc_pot = HernquistPotential(amp=smc_M, a=smc_a, ro=8., vo=220.)
smc_orbit = Orbit(vxvv=[16.26, -72.42, 62.8, 0.797, -1.220, 145.6], radec=True, ro=8.,vo=220.) 
smc_orbit.integrate(ts, MWPotential2014, dt=dt)
smc_potential = MovingObjectPotential(smc_orbit, pot=smc_pot)

# FORNAX
fnx_M = 2E9*u.Msun # https://academic.oup.com/mnras/article/368/3/1073/1022509
fnx_a = 3.4*u.kpc  # https://academic.oup.com/mnras/article-pdf/368/3/1073/18665310/mnras0368-1073.pdf
fnx_pot = HernquistPotential(amp=fnx_M, a=fnx_a)
fnx_orbit = Orbit(vxvv=[39.962, -34.511, 139.3, 0.374, -0.401, 55.3], radec=True, ro=8.,vo=220.) 
fnx_orbit.integrate(ts, MWPotential2014, dt=dt)
fnx_potential = MovingObjectPotential(fnx_orbit, pot=fnx_pot)

# DRACO
dra_M = 6.3E9*u.Msun # https://iopscience.iop.org/article/10.1086/521543/pdf
dra_a = 7*u.kpc # https://iopscience.iop.org/article/10.1086/521543/pdf
dra_pot = HernquistPotential(amp=dra_M, a=dra_a)
dra_orbit = Orbit(vxvv=[260.06, 57.965, 79.07, -0.012, -0.158, -291], radec=True, ro=8.,vo=220.) 
dra_orbit.integrate(ts, MWPotential2014, dt=dt)
dra_potential = MovingObjectPotential(dra_orbit, pot=dra_pot)

# URSA MINOR
umi_M = 2.5E9*u.Msun # https://iopscience.iop.org/article/10.1086/521543/pdf
umi_a = 5.4*u.kpc    # https://iopscience.iop.org/article/10.1086/521543/pdf
umi_pot = HernquistPotential(amp=umi_M, a=umi_a)
umi_orbit = Orbit(vxvv=[227.242, 67.222, 75.86, -0.184, 0.082, -246.9], radec=True, ro=8.,vo=220.) 
umi_orbit.integrate(ts, MWPotential2014, dt=dt)
umi_potential = MovingObjectPotential(umi_orbit, pot=umi_pot)

satellites_pot = [sgr_potential, lmc_potential, smc_potential, fnx_potential, dra_potential, umi_potential]
total_pot = [sgr_potential, lmc_potential, smc_potential, fnx_potential, dra_potential, umi_potential, MWPotential2014]

# Define ClusterArray object

In [5]:
class ClusterArray:
    def __init__(self, time=-12., steps=1000, include_Sgr=False, method='symplec4_c'):
        self.time = time
        self.steps = steps
        self.include_Sgr = include_Sgr
        self.method = method
        self.clusters = []
        self.vxvvs = []
        self.names = []
        if include_Sgr:
            self.potential = total_pot
        else:
            self.potential = MWPotential2014
    
    def load_from_names(self, names):
        self.names = names
        self.orbits = Orbit.from_name(names, solarmotion=[-11.1,24.,7.25])
        self.vxvvs = self.orbits.vxvv
        self.ctrl_orbits = Orbit.from_name(names, solarmotion=[-11.1,24.,7.25])
        self.integrate_orbits()
    
    def integrate_orbits(self):
        # Integrate orbit and save values
        ts=np.linspace(0, self.time/bovy_conversion.time_in_Gyr(ro=8.,vo=220.), self.steps)
        self.orbits.integrate(ts, self.potential, method=self.method, dt=dt)
        self.ctrl_orbits.integrate(ts, MWPotential2014, method=self.method, dt=dt)
    
    def get_rs(self, times=np.array([0])):
        o = self.orbits.getOrbit()
        index = abs(times/self.time*self.steps)
        index = index.astype(int)
        pars = o[:, index]
        new_orbit = Orbit(vxvv=pars, ro=8.,vo=220., solarmotion=[-11.1,24.,7.25])
        ts=np.linspace(0,-12.,self.steps)*u.Gyr # Gyr
        potential = deepcopy(total_pot)
        new_orbit.integrate(ts, potential, dt=dt)
        bovy_t = times/bovy_conversion.time_in_Gyr(ro=8, vo=220)
        times = np.linspace(0, -3./bovy_conversion.time_in_Gyr(ro=8.,vo=220.), self.steps)
        return np.array([new_orbit.R(t) for t in times])
    
    def get_orbits(self, times=np.array([0]), ctrl=False):
        o = self.ctrl_orbits.getOrbit() if ctrl else self.orbits.getOrbit()

        index = abs(times/self.time*self.steps)
        index = index.astype(int)
        pars = o[:, index]
        new_orbit = Orbit(vxvv=pars, ro=8.,vo=220., solarmotion=[-11.1,24.,7.25])
        #ts=np.linspace(0,-6.,self.steps)*u.Gyr # Gyr
        #new_orbit.integrate(ts, self.potential)
        return new_orbit

# Load objects and get orbits

In [6]:
t_max = -12.
times = np.array([0, -1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -11.999])
n_steps = 1201

In [7]:
names = ["ngc104", "ngc288", "ngc362", "whiting1", "ngc1261", "pal1", "e1", "eridanus", "pal2", "ngc1851", "ngc1904", "ngc2298", "ngc2419", "pyxis", "ngc2808", "e3", "pal3", "ngc3201", "pal4", "crater", "ngc4147", "ngc4372", "rup106", "ngc4590", "ngc4833", "ngc5024", "ngc5053", "ngc5139", "ngc5272", "ngc5286", "ngc5466", "ngc5634", "ngc5694", "ic4499", "ngc5824", "pal5", "ngc5897", "ngc5904", "ngc5927", "ngc5946", "bh176", "ngc5986", "fsr1716", "pal14", "bh184", "ngc6093", "ngc6121", "ngc6101", "ngc6144", "ngc6139", "terzan3", "ngc6171", "eso452-11", "ngc6205", "ngc6229", "ngc6218", "fsr1735", "ngc6235", "ngc6254", "ngc6256", "pal15", "ngc6266", "ngc6273", "ngc6284", "ngc6287", "ngc6293", "ngc6304", "ngc6316", "ngc6341", "ngc6325", "ngc6333", "ngc6342", "ngc6356", "ngc6355", "ngc6352", "ic1257", "terzan2", "ngc6366", "terzan4", "bh229", "ngc6362", "ngc6380", "terzan1", "ton2", "ngc6388", "ngc6402", "ngc6401", "ngc6397", "pal6", "ngc6426", "djorg1", "terzan5", "ngc6440", "ngc6441", "terzan6", "ngc6453", "ngc6496", "terzan9", "djorg2", "ngc6517", "terzan10", "ngc6522", "ngc6535", "ngc6528", "ngc6539", "ngc6540", "ngc6544", "ngc6541", "eso280-06", "ngc6553", "ngc6558", "pal7", "terzan12", "ngc6569", "bh261", "ngc6584", "ngc6624", "ngc6626", "ngc6638", "ngc6637", "ngc6642", "ngc6652", "ngc6656", "pal8", "ngc6681", "ngc6712", "ngc6715", "ngc6717", "ngc6723", "ngc6749", "ngc6752", "ngc6760", "ngc6779", "terzan7", "pal10", "arp2", "ngc6809", "terzan8", "pal11", "ngc6838", "ngc6864", "ngc6934", "ngc6981", "ngc7006", "ngc7078", "ngc7089", "ngc7099", "pal12", "pal13", "ngc7492"]

In [ ]:
ca = ClusterArray(include_Sgr=True, time=t_max, steps=n_steps)
ca_ctrl = ClusterArray(include_Sgr=False, time=t_max, steps=n_steps)
ca.load_from_names(names)
ca_ctrl.load_from_names(names)

# Pickle the ClusterArrays
Integrating the orbits takes a while, so pickle them

In [ ]:
with open('ca.pickle', 'wb') as handle:
    pickle.dump(ca, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('ca_ctrl.pickle', 'wb') as handle:
    pickle.dump(ca_ctrl, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Load the ClusterArrays from pickles

In [ ]:
with open('ca.pickle', 'rb') as handle:
    ca = pickle.load(handle)
with open('ca_ctrl.pickle', 'rb') as handle:
    ca_ctrl = pickle.load(handle)

# Get orbits at timesteps

In [ ]:
os = ca.get_orbits(times)
os_ctrl = ca_ctrl.get_orbits(times)

ca_os = ca.orbits
ca_os_ctrl = ca_ctrl.orbits

# Classify GCs

In [ ]:
def classify(os):
    classes = ['' for i in range(len(os))]
    for i in range(len(os)):
        if os[i, 0].rap(pot=MWPotential2014, analytic=True) < 3.5:
            classes[i] = 'bulge'
        elif os[i, 0].zmax(pot=MWPotential2014, analytic=True) < 5:
            classes[i] = "disk"
        else:
            classes[i] = "halo"
    return classes

In [ ]:
classes = classify(os)

# Plot eccentricity difference

In [ ]:
e_0 = os_ctrl[:, 0].e(pot=MWPotential2014, analytic=True)
e_f = os_ctrl[:, -1].e(pot=MWPotential2014, analytic=True)

In [ ]:
fig, ax = plt.subplots()
for ind in range(len(os)):
    if classes[ind] == 'bulge':
        ax.plot(e_0[ind], e_f[ind], 'o', color='b', label='bulge')
    elif classes[ind] == 'disk':
        ax.plot(e_0[ind], e_f[ind], 'o', color='r', label='disk')
    elif classes[ind] == 'halo':
        ax.plot(e_0[ind], e_f[ind], 'o', color='g', label='halo')

ax.set_xlabel("Eccentricity at t=0")
ax.set_ylabel("Eccentricity at t=-12")
ax.plot([0, 1.1], [0, 1.1], 'k--')
ax.set_xlim([0, 1.1])
ax.set_ylim([0, 1.1])
plt.show()

# Plot tidal radius difference

In [ ]:
# rt = gp.potential.rtide(MWPotential2014, R=os.R(), z=os.z(), phi=os.phi(), M=10E5*u.Msun)*8
rt_0 = gp.potential.rtide(MWPotential2014, R=ca_os_ctrl.R(0), z=ca_os_ctrl.z(0), phi=ca_os_ctrl.phi(0), M=1E6*u.Msun)
rt_f = gp.potential.rtide(MWPotential2014, R=ca_os_ctrl.R(-12*u.Gyr), z=ca_os_ctrl.z(-12*u.Gyr), phi=ca_os_ctrl.phi(-12*u.Gyr), M=1E6*u.Msun)

In [ ]:
fig, ax = plt.subplots()
for ind in range(len(os)):
    if classes[ind] == 'bulge':
        ax.plot(rt_0[ind], rt_f[ind], 'o', color='b', label='bulge')
    elif classes[ind] == 'disk':
        ax.plot(rt_0[ind], rt_f[ind], 'o', color='r', label='disk')
    elif classes[ind] == 'halo':
        ax.plot(rt_0[ind], rt_f[ind], 'o', color='g', label='halo')

ax.set_xlabel("$R_{tide}$, t=0")
ax.set_ylabel("$R_{tide}$, t=-12")
ax.plot([0, 1.], [0, 1.], 'k--')
plt.show()

# Plot eccentricity vs semi-major axis for all clusters

In [ ]:
rap = os.rap(analytic=True, pot=MWPotential2014)
rperi = os.rperi(analytic=True, pot=MWPotential2014)
eccen = (rap-rperi)/(rap+rperi)#os_ctrl.e(analytic=True)

for i in range(len(rap)):
    x = 0
    y = 0
    for j in range(len(rap[0])):
        if not np.isinf(rap[i][j]):
            x = rap[i][j]
        else:
            rap[i][j] = x
        if not np.isneginf(rperi[i][j]):
            y = rperi[i][j]
        else:
            rperi[i][j] = y
a = (rap+rperi)/2.
loga = np.log10(a)

In [ ]:
rt_diff = rt_0-rt_f
rt_diff_norm = (rt_diff-min(rt_diff))/(max(rt_diff)-min(rt_diff))

In [ ]:
name_choices = ["ngc104", "ngc288", "ngc362", "whiting1", "ngc1261", "pal1", "e1", "eridanus", "pal2", "ngc1851", "ngc1904", "ngc2298", "ngc2419", "pyxis", "ngc2808", "e3", "pal3", "ngc3201", "pal4", "crater", "ngc4147", "ngc4372", "rup106", "ngc4590", "ngc4833", "ngc5024", "ngc5053", "ngc5139", "ngc5272", "ngc5286", "ngc5466", "ngc5634", "ngc5694", "ic4499", "ngc5824", "pal5", "ngc5897", "ngc5904", "ngc5927", "ngc5946", "bh176", "ngc5986", "fsr1716", "pal14", "bh184", "ngc6093", "ngc6121", "ngc6101", "ngc6144", "ngc6139", "terzan3", "ngc6171", "eso452-11", "ngc6205", "ngc6229", "ngc6218", "fsr1735", "ngc6235", "ngc6254", "ngc6256", "pal15", "ngc6266", "ngc6273", "ngc6284", "ngc6287", "ngc6293", "ngc6304", "ngc6316", "ngc6341", "ngc6325", "ngc6333", "ngc6342", "ngc6356", "ngc6355", "ngc6352", "ic1257", "terzan2", "ngc6366", "terzan4", "bh229", "ngc6362", "ngc6380", "terzan1", "ton2", "ngc6388", "ngc6402", "ngc6401", "ngc6397", "pal6", "ngc6426", "djorg1", "terzan5", "ngc6440", "ngc6441", "terzan6", "ngc6453", "ngc6496", "terzan9", "djorg2", "ngc6517", "terzan10", "ngc6522", "ngc6535", "ngc6528", "ngc6539", "ngc6540", "ngc6544", "ngc6541", "eso280-06", "ngc6553", "ngc6558", "pal7", "terzan12", "ngc6569", "bh261", "ngc6584", "ngc6624", "ngc6626", "ngc6638", "ngc6637", "ngc6642", "ngc6652", "ngc6656", "pal8", "ngc6681", "ngc6712", "ngc6715", "ngc6717", "ngc6723", "ngc6749", "ngc6752", "ngc6760", "ngc6779", "terzan7", "pal10", "arp2", "ngc6809", "terzan8", "pal11", "ngc6838", "ngc6864", "ngc6934", "ngc6981", "ngc7006", "ngc7078", "ngc7089", "ngc7099", "pal12", "pal13", "ngc7492"]
#name_choices = ['eso280-06', "ngc4590", "ngc5466", "ngc5824", "ngc6712", "ngc6838","ngc2298","ngc6218"]
choices = [names.index(i) for i in name_choices]

In [ ]:
fig, ax = plt.subplots()

plt.plot(np.log10((sgr_orbit.rap()+sgr_orbit.rperi())/2), sgr_orbit.e(), 'x', markersize=9, c='#888888', mew=2)
plt.plot(np.log10((lmc_orbit.rap()+lmc_orbit.rperi())/2), lmc_orbit.e(), 'x', markersize=9, c='#888888', mew=2)
plt.plot(np.log10((smc_orbit.rap()+smc_orbit.rperi())/2), smc_orbit.e(), 'x', markersize=9, c='#888888', mew=2)
plt.plot(np.log10((fnx_orbit.rap()+fnx_orbit.rperi())/2), fnx_orbit.e(), 'x', markersize=9, c='#888888', mew=2)
plt.plot(np.log10((dra_orbit.rap()+dra_orbit.rperi())/2), dra_orbit.e(), 'x', markersize=9, c='#888888', mew=2)
plt.plot(np.log10((umi_orbit.rap()+umi_orbit.rperi())/2), umi_orbit.e(), 'x', markersize=9, c='#888888', mew=2)

cmap = LinearSegmentedColormap.from_list('mycmap', [(0.0, '#3b4cc0'),
                                                    (0.1, '#4f69d9'),
                                                    (0.3, '#80a3fa'),
                                                    (0.5, '#b2ccfb'),
                                                    (0.7, '#dcdddd'),
                                                    (0.9, '#f6bfa6'),
                                                    (1.0, '#f6a385')
                                                    ])
sc = ax.scatter(loga[:, -1], eccen[:, -1], marker='o', c=rt_diff, cmap=cmap)

for ind in choices:
    ax.plot(loga[ind, -1], eccen[ind, -1], markersize=11, marker='o', c='r', mfc='none')

for ind in range(len(rap)):
    c = cmap((rt_diff[ind]-min(rt_diff))/(max(rt_diff)-min(rt_diff)))
    ax.arrow(loga[ind, -1], eccen[ind, -1], loga[ind, 0]-loga[ind, -1], eccen[ind, 0]-eccen[ind, -1],
             head_width=0.03, 
             head_length=0.03, 
             fc=c,
             ec=c,
             linewidth=0.7)   
    
ax.set_xlabel("$log($semi-major axis$)$", fontsize=12)
ax.set_ylabel("Eccentricity", fontsize=12)
ax.tick_params(labelsize=12)
cb = plt.colorbar(sc)
cb.set_label("Change in tidal radius (kpc)", rotation=270, fontsize=12, labelpad=14)
fig.savefig('eccen_loga.png', dpi=600)
plt.show()

# Plot clusters of choice

In [ ]:
fig, ax = plt.subplots()

name_choices = ["ngc288", "whiting1", "ngc1261", "ngc1851", "ngc2298", "ngc2419", "ngc2808", "e3", "ngc3201", "ngc4147", "rup106", "ngc4590", "ngc4833", "ngc5024", "ngc5053", "ngc5139", "ngc5272", "ngc5286", "ngc5466", "ngc5634", "ngc5694", "ic4499", "ngc5824", "pal5", "ngc5897", "ngc5904", "ngc5946", "ngc5986", "ngc6093", "ngc6121", "ngc6101", "ngc6144", "ngc6139", "terzan3", "ngc6171", "eso452-11", "ngc6205", "ngc6229", "ngc6218", "fsr1735", "ngc6235", "ngc6254", "ngc6256", "pal15", "ngc6266", "ngc6273", "ngc6284", "ngc6287", "ngc6293", "ngc6316", "ngc6325", "ngc6333", "ngc6342", "ngc6356", "ngc6355", "ic1257", "ngc6366", "bh229", "ngc6362", "ngc6380", "ton2", "ngc6388", "ngc6402", "ngc6401", "ngc6426", "ngc6441", "ngc6453", "ngc6496", "ngc6517", "terzan10", "ngc6522", "ngc6535", "ngc6528", "ngc6539", "ngc6544", "ngc6541", "ngc6558", "terzan12", "ngc6569", "ngc6584", "ngc6624", "ngc6626", "ngc6638", "ngc6637", "ngc6652", "ngc6656", "pal8", "ngc6681", "ngc6712", "ngc6715", "ngc6717", "ngc6723", "terzan7", "pal10", "arp2", "ngc6809", "terzan8", "ngc6864", "ngc6934", "ngc7006", "ngc7089", "ngc7099", "pal12", "ngc7492"]
choices = [names.index(i) for i in name_choices]

sc = ax.scatter(loga[:, -1][choices], eccen[:, -1][choices], marker='o', c='k')

for i, txt in enumerate(name_choices):
    plt.text(loga[choices[i], -1]+0.02, eccen[choices[i], -1]-0.02, txt, fontsize=6)

for ind in choices:
    ax.plot(loga[ind], eccen[ind], c='k')   
    
#     ax.arrow(loga[ind, -1], eccen[ind, -1], loga[ind, 0]-loga[ind, -1], eccen[ind, 0]-eccen[ind, -1],
#              head_width=0.03, 
#              head_length=0.03, 
#              fc='k',
#              ec='k',
#              linewidth=0.7)   

ax.set_xlabel("log(a)")
ax.set_ylabel("Eccentricity")
ax.set_title("Change in globular cluster orbit over 12 Gyr")
fig.savefig('eccen_loga.png')
plt.show()

# Plot SMA at start and end

In [ ]:
SMA = (rap+rperi)/2

In [ ]:
fig, ax = plt.subplots()

ax.plot(rap[:, 0], rap[:, -1], 'o')
ax.set_xlabel("Apocentre at t=0")
ax.set_ylabel("Apocentre at t=-12")
ax.plot([0, 300], [0, 300], 'k--')
plt.show()
print("apo_12-apo_0", sum(rap[:, -1]-rap[:, 0])) 

In [ ]:
fig, ax = plt.subplots()

ax.plot(rperi[:, 0], rperi[:, -1], 'o')
ax.set_xlabel("Pericentre_0")
ax.set_ylabel("Pericentre_f")
ax.plot([0, 140], [0, 140], 'k--')
plt.show()
print("peri_12-peri_0:", sum(rperi[:, -1]-rperi[:, 0]))

In [ ]:
fig, ax = plt.subplots()

ax.plot(SMA[:, 0], SMA[:, -1], 'o')
ax.set_xlabel("SMA_0")
ax.set_ylabel("SMA_f")
ax.plot([0, 175], [0, 175], 'k--')
plt.show()
print("SMA_12-SMA_0:", sum(SMA[:, -1]-SMA[:, 0]))

# Plot action angle coordinates for all clusters

In [ ]:
jr = os.jr(pot=MWPotential2014)
jp = os.jp(pot=MWPotential2014)
jz = os.jz(pot=MWPotential2014)

In [ ]:
fig, ax = plt.subplots()
for ind in range(len(jr)):
    l = ax.plot(jp[ind], jr[ind])
    ax.plot(jp[ind, 0], jr[ind, 0], 'o', color=l[0].get_color())

ax.set_xlabel("$J_{\phi}$")
ax.set_ylabel("$J_r$")
plt.show()

In [ ]:
fig, ax = plt.subplots()
for ind in range(len(jr)):
    l = ax.plot(jz[ind], jr[ind])
    ax.plot(jz[ind, 0], jr[ind, 0], 'o', color=l[0].get_color())

ax.set_xlabel("$J_z$")
ax.set_ylabel("$J_r$")
plt.show()

In [ ]:
fig, ax = plt.subplots()
for ind in range(len(jr)):
    l = ax.plot(jp[ind], jz[ind])
    ax.plot(jp[ind, 0], jz[ind, 0], 'o', color=l[0].get_color())

ax.set_xlabel("$J_{\phi}$")
ax.set_ylabel("$J_z$")
plt.show()